In [58]:
import pymongo
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from pymongo import MongoClient
import time
from datetime import datetime


def connect_mongo_and_return_collections(login, password, IP, port, database_name, collections_name):
    """
    Nawiązuje połączenie z bazą danych MongoDB i zwraca kolekcję do operacji.

    :param login: Login użytkownika do MongoDB.
    :param password: Hasło użytkownika do MongoDB.
    :param IP: Adres IP serwera MongoDB.
    :param port: Port serwera MongoDB.
    :param database_name: Nazwa bazy danych, do której chcemy się połączyć.
    :param collections_name: Nazwa kolekcji w bazie danych, z którą chcemy pracować.
    :return: Obiekt kolekcji MongoDB umożliwiający wykonywanie operacji na danych.
    """
    conn = pymongo.MongoClient(f'mongodb://{login}:{password}@{IP}:{port}/')
    db = conn[f'{database_name}']
    matches_coll = db[f'{collections_name}']

    try:
        # Test połączenia
        conn.admin.command('ping')
        print("Połączenie z MongoDB nawiązane!")
    except Exception as e:
        print(f"Nie udało się połączyć z MongoDB: {e}")
        exit()

    return matches_coll


def get_player_delta_elo(player_name, n):
    """
    Pobiera dane dotyczące zmian rankingu ELO dla gracza z ostatnich n turniejów.

    :param player_name: Nazwa gracza, którego dane mają być analizowane.
    :param n: Liczba ostatnich turniejów do uwzględnienia w analizie.
    :return: Lista słowników zawierających datę turnieju i skumulowaną wartość `delta_elo`.

    Kluczowe kroki:
      1. Filtruje mecze, w których gracz występował jako gospodarz (`home_name`) lub gość (`away_name`).
      2. Grupuje dane według turnieju i daty, sumując zmiany ELO (`delta_elo`).
         Uwzględnia znak zmiany w zależności od roli gracza (gospodarz/gość).
      3. Sortuje wyniki po dacie i ogranicza do maksymalnie `n` najnowszych turniejów.

    Funkcja zwraca wyniki w postaci listy, gdzie każdy element
    zawiera datę turnieju oraz całkowitą zmianę `delta_elo` w danym turnieju.
    """
    pipeline = [
        {"$match": {
            "$or": [
                {"home_name": player_name},
                {"away_name": player_name}
            ]
        }},
        {"$sort": {"datetime": -1}},
        {"$group": {
            "_id": {
                "league_name": "$league_name",
                "date": {"$dateToString": {"format": "%Y-%m-%d", "date": "$datetime"}}
            },
            "date": {"$first": "$datetime"},
            "total_delta_elo": {
                "$sum": {
                    "$cond": {
                        "if": {"$eq": ["$home_name", player_name]},
                        "then": "$delta_elo",
                        "else": {"$multiply": ["$delta_elo", -1]}
                    }
                }
            }
        }},
        {"$sort": {"date": -1}},
        {"$limit": n}
    ]
    matches_coll = connect_mongo_and_return_collections(
        'root', 'Awusawus!', '192.168.0.13', '27017',
        'czech_liga_pro_test', 'matches'
    )

    # Pomiar czasu rozpoczęcia
    start_time = time.time()
    last_n_tournaments = list(matches_coll.aggregate(pipeline))
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Czas wykonania zapytania: {execution_time:.6f} sekund")

    if not last_n_tournaments:
        return "Brak wyników zapytania."
    else:
        return last_n_tournaments


def return_trend(delta_elo, n_values):
    """
    Oblicza współczynniki nachylenia (slope) dla różnych wartości n ostatnich turniejów.

    :param delta_elo: Lista zmian delta_elo z turniejów (najbardziej aktualne jako pierwsze).
    :param n_values: Lista n, dla których chcemy obliczyć regresję (np. [100, 50, 20, 10]).
    :return: Słownik, w którym kluczami są wartości n, a wartościami są współczynniki nachylenia.
    """
    trends = {}

    for n in n_values:
        # Bierzemy n ostatnich elementów (lub wszystkie, jeśli jest ich mniej niż n)
        limited_delta_elo = delta_elo[:n]

        # Odwracamy listę, aby najstarszy mecz był pierwszym punktem
        delta_elo_reversed = limited_delta_elo[::-1]

        # Tworzymy tablicę x jako indeksy meczów (np. [0, 1, 2, ...])
        x = np.arange(len(delta_elo_reversed))

        # Dodajemy punkt początkowy (0 ELO) na początek listy
        cumulative_elo = np.cumsum(np.insert(delta_elo_reversed, 0, 0))

        # Indeksy meczów, do których mamy skumulowane ELO
        x = np.arange(len(cumulative_elo))  # Zaktualizowane indeksy z punktem początkowym

        # Obliczamy współczynnik kierunkowy (nachylenie) prostej regresji
        slope, _, _, _, _ = stats.linregress(x, cumulative_elo)

        # Dodajemy wynik do słownika
        trends[n] = slope

    return trends


def main(player_name):
    """
    Główna funkcja wykonująca skrypt.
    """
    
    n_values = [100, 50, 20, 10]  # Wartości n-liczba ostatnich turniejów do analizy

    start_time = time.time()
    # Pobranie danych o ostatnich turniejach
    last_n_tournaments = get_player_delta_elo(player_name, max(n_values))

    # Wyciągnięcie wartości delta_elo z danych
    delta_elo = [tournament['total_delta_elo'] for tournament in last_n_tournaments]

    # Obliczenie trendów
    trends = return_trend(delta_elo, n_values)

    # Wyświetlenie wyników
    print("Trendy formy zawodnika:")
    for n, slope in trends.items():
        print(f"Dla ostatnich {n} turniejów: nachylenie = {slope:.2f}")

    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Czas wykonania skryptu: {execution_time:.6f} sekund")


if __name__ == "__main__":
    main('Raska M')


Połączenie z MongoDB nawiązane!
Czas wykonania zapytania: 0.073112 sekund
Trendy formy zawodnika:
Dla ostatnich 100 turniejów: nachylenie = 0.26
Dla ostatnich 50 turniejów: nachylenie = 0.21
Dla ostatnich 20 turniejów: nachylenie = 0.35
Dla ostatnich 10 turniejów: nachylenie = 1.63
Czas wykonania skryptu: 0.402607 sekund
